# Set references for v10 tuning and export to ONNX

This notebook is dedicated to apply the linear correction in the neural network output w.r.t the avgmu and export the v9 best models to ONNX/keras format. Usually, keras versions is used into the prometheus framework. The ONNX version will be used into the athena framework.

**NOTE**: ONNX is a Microsoft API for inference.

**NOTE**: We will export all tunings from v10 r2 derivation

In [1]:
from kolmov import crossval_table, get_color_fader, fit_table
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
%config InlineBackend.figure_format = 'retina'

Welcome to JupyROOT 6.23/01
INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Using all sub packages with ROOT dependence


In [2]:
def create_op_dict(op):
    d = {
              op+'_pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              op+'_fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              op+'_sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              op+'_pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              op+'_fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              op+'_sp_val'    : "reference/"+op+"_cutbased/sp_val",
              op+'_pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              op+'_fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              op+'_sp_op'     : "reference/"+op+"_cutbased/sp_op",

              # Counts
              op+'_pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              op+'_fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              op+'_pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              op+'_fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",
              op+'_pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              op+'_fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              op+'_pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              op+'_fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",
              op+'_pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              op+'_fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              op+'_pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              op+'_fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    }
    return d

tuned_info = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              } )

tuned_info.update(create_op_dict('tight'))
tuned_info.update(create_op_dict('medium'))
tuned_info.update(create_op_dict('loose'))
tuned_info.update(create_op_dict('vloose'))

In [3]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:


In [4]:
cv  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [5]:
cv.fill(  '/Volumes/castor/tuning_data/Zee/v10/r2/*/*/*.gz', 'v10')

2021-04-13 22:45:36,472 | Py.crossval_table                       INFO Reading file for v10 tag from /Volumes/castor/tuning_data/Zee/v10/r2/*/*/*.gz
2021-04-13 22:45:36,473 | Py.crossval_table                       INFO There are 2500 files for this task...
2021-04-13 22:45:36,473 | Py.crossval_table                       INFO Filling the table... 
2021-04-13 22:46:48,065 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


### 1.1) Get best inits and sorts:

In [6]:
best_inits = cv.filter_inits("max_sp_val")
best_sorts = cv.filter_sorts( best_inits , 'max_sp_op')

### 1.2) Get best models:

Get all best models for each bin. Expected to be 25 models.

In [7]:
best_models = cv.get_best_models(best_sorts, remove_last=True)

In [8]:
best_models[0][0]['model'].summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 100)]             0         
_________________________________________________________________
Reshape_layer (Reshape)      (None, 100, 1)            0         
_________________________________________________________________
conv1d_layer_1 (Conv1D)      (None, 99, 4)             12        
_________________________________________________________________
conv1d_layer_2 (Conv1D)      (None, 98, 8)             72        
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_layer (Dense)          (None, 16)                12560     
_________________________________________________________________
output_for_inference (Dense) (None, 1)                 17    

In [15]:
import tensorflow as tf
model = cv.get_best_models(best_sorts, remove_last=False)[0][0]['model']
tf.keras.utils.plot_model(model, to_file='model_v10_plot.pdf', 
                          show_shapes=True, 
                          show_layer_names=True,
                          show_dtype=False,
                          rankdir='TB',
                          dpi=200)

## 2) Linear correction:

Here we will set all thresholds to operate as the same pd reference from cut-based using the pileup linear correction strategy. As the classifier efficiency has some "dependence" w.r.t the pileup we adopt the linear adjustment to "fix" the trigger efficiency. Here we will "fix" the neural network threshold w.r.t the pileup. 

### 2.1) Get all PD/Fas values:

Read all reference values from the storage.

In [9]:
# calculate all pd/fa from reference file
ref_path = '/Volumes/castor/cern_data/files/Zee/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97/references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.ref.pic.gz'
ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]
ref_matrix = [[ {} for eta in range(5)] for et in range(5)]
references = ['tight_cutbased', 'medium_cutbased' , 'loose_cutbased', 'vloose_cutbased']
from saphyra.core import ReferenceReader
for et_bin in range(5):
    for eta_bin in range(5):
        for name in references:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            pd = refObj.getSgnPassed(name)/refObj.getSgnTotal(name)
            fa = refObj.getBkgPassed(name)/refObj.getBkgTotal(name)
            ref_matrix[et_bin][eta_bin][name] = {'pd':pd, 'fa':fa}

### 2.2) Create data generator:

Since each tuning models is fed by a different data organization, we need to create a generator to open the data file, prepare the matrix and apply some pre-processing (if needed).

In [10]:
def generator( path ):
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    from Gaugi import load
    d = load(path)
    feature_names = d['features'].tolist()

    # How many events?
    n = d['data'].shape[0]
    
    # extract rings
    data_rings = norm1(d['data'][:,1:101])
    target = d['target']
    avgmu = d['data'][:,0]
    
    return [data_rings], target, avgmu

In [11]:
path = '/Volumes/castor/cern_data/files/Zee/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.npz'
paths = [[ path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]

In [12]:
# create the table class
ct  = fit_table( generator, etbins , etabins, 0.02, 0.5, 16, 60, xmin_percentage=0.05, xmax_percentage=99.95 )

### 2.3) Apply linear correction:

**NOTE**: Take about 25 minutes.

In [13]:
# Fill it
ct.fill(paths, best_models, ref_matrix,'correction_v10_data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose')


Applying ATLAS style settings...
Fitting... |############################################################| 25/25
Fitting... ... finished task in 1925.086415s.
2021-03-06 13:48:01,182 | Py.fit_table                         WARNING Retrieved positive angular factor of the linear correction, setting to 0!
2021-03-06 13:48:19,588 | Py.fit_table                         WARNING Retrieved positive angular factor of the linear correction, setting to 0!
2021-03-06 13:48:56,696 | Py.fit_table                         WARNING Retrieved positive angular factor of the linear correction, setting to 0!
2021-03-06 13:49:11,534 | Py.fit_table                         WARNING Retrieved positive angular factor of the linear correction, setting to 0!
2021-03-06 13:49:16,371 | Py.fit_table                         WARNING Retrieved positive angular factor of the linear correction, setting to 0!
2021-03-06 13:49:21,232 | Py.fit_table                         WARNING Retrieved positive angular factor of the lin

In [14]:
ct.table().head()

,name,et_bin,eta_bin,reference_signal_passed,reference_signal_total,reference_signal_eff,reference_background_passed,reference_background_total,reference_background_eff,signal_passed,...,signal_eff,background_passed,background_total,background_eff,signal_corrected_passed,signal_corrected_total,signal_corrected_eff,background_corrected_passed,background_corrected_total,background_corrected_eff
0,tight_cutbased,0,0,227619,232819,0.977666,23318,187639,0.124271,227573,...,0.977467,3467,187639,0.018477,227566,232819,0.977437,3380,187639,0.018013
1,medium_cutbased,0,0,227780,232819,0.978360,24336,187639,0.129701,227754,...,0.978245,3534,187639,0.018834,227725,232819,0.978120,3442,187639,0.018344
2,loose_cutbased,0,0,229996,232819,0.987876,31867,187639,0.169837,229978,...,0.987797,4974,187639,0.026508,229960,232819,0.987720,4783,187639,0.025490
3,vloose_cutbased,0,0,230152,232819,0.988548,32748,187639,0.174527,230132,...,0.988459,5128,187639,0.027329,230139,232819,0.988489,4948,187639,0.026370
4,tight_cutbased,0,1,137861,141000,0.977742,31938,143657,0.222321,137838,...,0.977574,4505,143657,0.031359,137841,141000,0.977596,4379,143657,0.030482


### 2.3) Create beamer report:

In [15]:
ct.dump_beamer_table(ct.table(), best_models, 'data17_13TeV v10 tuning', 
                                              'correction_v10_data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.pdf')

2021-03-06 13:49:27,829 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file correction_v10_data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.pdf latex code...


## 3) Export all tunings:

In [16]:
model_name_format = 'data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'
for idx, op in enumerate(['Tight','Medium','Loose','VeryLoose']):
    ct.export(best_models, 
              model_name_format.format(op=op), 
              config_name_format.format(op=op), 
              references[idx], 
              to_onnx=True)

Saving ONNX file as models/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electronTight.et0_eta0.onnx
Saving ONNX file as models/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electronTight.et0_eta1.onnx
Saving ONNX file as models/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electronTight.et0_eta2.onnx
Saving ONNX file as models/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electronTight.et0_eta3.onnx
Saving ONNX file as models/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electronTight.et0_eta4.onnx
Saving ONNX file as models/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electronTight.et1_eta0.onnx
Saving ONNX file as models/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electronTight.et1_eta1.onnx
Saving ONNX file as models/data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v10.electronTight.et1_eta2.onnx
Saving ONNX file as models/data17_13TeV_EGAM1_pr

2021-03-06 13:49:50.055405: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-03-06 13:49:50.092947: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9dd4c56530 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-03-06 13:49:50.092963: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 20 -> 12
The maximum opset needed by this model is only 11.
2021-03-06 13:49:53.104717: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (one